# NN Model
#### cv score :  0.5432514177693761 (5 folds, model3)
#### cv score :  ?? (10 folds)

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro, NNModel
import warnings

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.callbacks import EarlyStopping


random.seed(0)

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("../../data/Processed/train3.csv.gz")
test_df = pd.read_csv("../../data/Processed/test3.csv.gz")
print(train_df.shape)
print(test_df.shape)

(132250, 944)
(28340, 943)


In [3]:
y = train_df["y"]
train_df = train_df.drop("y", axis=1)

In [4]:
# make input


drop_cols = ["id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", \
              "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]

categorical_feature = [col for col in train_df.dtypes[train_df.dtypes == "object"].index.to_list() if col not in drop_cols]

X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=True, scaler=True, verbose=False)

#X[categorical_feature] = X[categorical_feature].astype("category")

scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scale
scal

In [5]:
# 全データを5つに分割
random.seed(random.randint(0, 10000))
SIZE = X.shape[0]
K = 5
cat_tgtenc_cols = ["mode", "stage", "team-category1-A", "team-category1-B"]
#cat_tgtenc_cols = ["mode", "stage"]

folds = prepro.make_stratified_kfolds(X, y, K, shuffle=True)
print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    
    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")

5
fold  1  size is  26450
fold  2  size is  26450
fold  3  size is  26450
fold  4  size is  26450
fold  5  size is  26450
successfully split


In [ ]:
THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []

all_indices = sum(folds, [])
for i in range(K):
    print("======================== fold {} ========================".format(i+1))
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    train_X = X.iloc[train_indices]
    train_y = y.iloc[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y.iloc[valid_indices]
    
    
    model = NNModel.make_model2(X, test_X, categorical_feature)
    #model = NNModel.make_model(X, test_X, categorical_feature)
    
    
    
    train_X = NNModel.prepro_nn(train_X, categorical_feature)
    train_y = train_y.values.reshape(-1, 1)
    valid_X = NNModel.prepro_nn(valid_X, categorical_feature)
    valid_y = valid_y.values.reshape(-1, 1)
    
    
    

    cb = EarlyStopping(monitor='val_loss', patience=1)

    model.fit(
        train_X, 
        train_y,
        batch_size=256,
        epochs=10,
        verbose=1,
        callbacks=[cb],
        validation_data=(valid_X, valid_y),
    )
    
    pred = model.predict(valid_X)
    pred = np.where(pred < THRESHOLD, 0, 1)
    train_pred.append(pred)
    temp += np.sum(pred)
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

======================== fold 1 ========================
Train on 105800 samples, validate on 26450 samples
Epoch 1/10
105800/105800 [==============================] - 144s 1ms/sample - loss: 0.6996 - val_loss: 0.6906
Epoch 2/10
105800/105800 [==============================] - 115s 1ms/sample - loss: 0.6901 - val_loss: 0.6876
Epoch 3/10
105800/105800 [==============================] - 102s 960us/sample - loss: 0.6879 - val_loss: 0.6860
Epoch 4/10
105800/105800 [==============================] - 111s 1ms/sample - loss: 0.6861 - val_loss: 0.6853
Epoch 5/10
105800/105800 [==============================] - 115s 1ms/sample - loss: 0.6846 - val_loss: 0.6844
Epoch 6/10
105800/105800 [==============================] - 119s 1ms/sample - loss: 0.6804 - val_loss: 0.6846


In [ ]:
preds = []
test_X = NNModel.prepro_nn(test_X, categorical_feature)

for i in range(K):
    model = models[i]
    pred = model.predict(test_X, batch_size=128).reshape(-1, )
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":preds})
temp.hist(bins=20)

In [ ]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_NN_3_{}.csv'.format(K))

# モデル解釈

In [ ]:
importance = pd.DataFrame(models[1].feature_importance(), index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance[:50])

In [ ]:
importance["importance"].sum()

In [ ]:
train_df["pred"] = 0
train_df["y"] = y.values
for i in range(K):
    train_df["pred"].iloc[folds[i]] = train_pred[i].reshape(-1, )

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df

In [ ]:
import itertools
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
import category_encoders.target_encoder as te

import random

def make_input(df1, df2, drop_col, categorical_encode, scaler, verbose):

    all_df = pd.concat([df1, df2])
    df1 = df1.drop(columns=drop_col, axis=1)
    df2 = df2.drop(columns=drop_col, axis=1)
    cols = df1.columns
    for col in cols:
        if verbose:
            print(col)


        elif df2[col].dtype in [int, float]:
            if scaler and "onehot" not in col:
                df1[col] = df1[col].apply(np.log1p)
                df2[col] = df2[col].apply(np.log1p)
                scaler = StandardScaler()
                print("scale")
                scaler.fit(all_df[col].values.reshape(-1, 1))
                df1[col] = scaler.transform(df1[col].values.reshape(-1, 1))
                df2[col] = scaler.transform(df2[col].values.reshape(-1, 1))
            continue
        if categorical_encode:
            lbl = LabelEncoder()
            obj = all_df[col].unique()
            lbl.fit(obj)
            df1[col] = lbl.transform(df1[col])
            df2[col] = lbl.transform(df2[col])
    print("complete")
    return df1, df2